In [1]:
# importa os pacotes necessários
import numpy as np
import os, gc, cv2, random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

import warnings
warnings.filterwarnings("ignore")

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
import sklearn.metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import  cross_val_score

from sklearn import svm
from skimage.feature import greycomatrix, greycoprops
from skimage.feature import hog
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

from sklearn.svm import LinearSVC
from sklearn.svm import SVC

In [2]:
df = pd.read_csv('kaggle/data_pca.csv')
df_labels = pd.read_csv('kaggle/data_labels.csv')

In [3]:
#df=(df-df.min())/(df.max()-df.min())

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df, df_labels.label.values, test_size = 0.3, random_state = 42)
del df
X_train.shape,X_test.shape

((35000, 2500), (15000, 2500))

# Logistic Regression
Utilizando regressão logistica para ter uma base de comparação com os demais modelos.

#### nomalizado antes e padding

In [5]:
%%time
logreg = LogisticRegression(solver='saga',n_jobs=2)
logreg.fit(X_train, y_train)

Wall time: 8min 42s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=2, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [6]:
# avalia predicao
y_pred = logreg.predict(X_test)
report = classification_report(y_test, y_pred,output_dict = True)
pd.DataFrame(report)

,airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck,accuracy,macro avg,weighted avg
f1-score,0.602136,0.633367,0.448193,0.376615,0.452621,0.439560,0.610732,0.611316,0.709432,0.627002,0.5532,0.551097,0.550901
precision,0.592252,0.623198,0.463345,0.395714,0.451524,0.452675,0.583230,0.610294,0.696818,0.624756,0.5532,0.549381,0.549338
recall,0.612356,0.643873,0.434000,0.359274,0.453723,0.427184,0.640956,0.612341,0.722510,0.629265,0.5532,0.553548,0.553200
support,1473.000000,1477.000000,1500.000000,1542.000000,1437.000000,1545.000000,1465.000000,1491.000000,1546.000000,1524.000000,0.5532,15000.000000,15000.000000


In [7]:
%%time
logreg = LogisticRegression(solver='saga',n_jobs=4)
logreg.fit(X_train.iloc[:,:1500], y_train)

Wall time: 4min 31s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=4, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [8]:
# avalia predicao
y_pred = logreg.predict(X_test.iloc[:,:1500])
report = classification_report(y_test, y_pred,output_dict = True)
pd.DataFrame(report)

,airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck,accuracy,macro avg,weighted avg
f1-score,0.583333,0.618705,0.422565,0.366985,0.455473,0.427373,0.588689,0.584270,0.688316,0.604092,0.536667,0.533980,0.533731
precision,0.577512,0.598355,0.445347,0.387050,0.460826,0.443593,0.556163,0.575896,0.677743,0.598071,0.536667,0.532056,0.531970
recall,0.589274,0.640487,0.402000,0.348898,0.450244,0.412298,0.625256,0.592891,0.699224,0.610236,0.536667,0.537081,0.536667
support,1473.000000,1477.000000,1500.000000,1542.000000,1437.000000,1545.000000,1465.000000,1491.000000,1546.000000,1524.000000,0.536667,15000.000000,15000.000000


# Linear SVM
Classificador SVM otimizado para o kernel linear, pode não dar o mesmo 
resultado se comparado ao sklearn.svm.SVC(), mas tem um tempo de execução menor

In [9]:
%%time
svclassifier = LinearSVC()
svclassifier.fit(X_train, y_train)

Wall time: 21min


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [10]:
# avalia predicao
y_pred = svclassifier.predict(X_test)
report = classification_report(y_test, y_pred,output_dict = True)
pd.DataFrame(report)

,airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck,accuracy,macro avg,weighted avg
f1-score,0.555868,0.608581,0.399725,0.347711,0.404295,0.393487,0.585232,0.575927,0.680680,0.584398,0.517067,0.513590,0.513424
precision,0.576111,0.593690,0.412181,0.355978,0.431962,0.413400,0.527382,0.563543,0.651865,0.599310,0.517067,0.512542,0.512407
recall,0.536999,0.624238,0.388000,0.339818,0.379958,0.375405,0.657338,0.588867,0.712160,0.570210,0.517067,0.517299,0.517067
support,1473.000000,1477.000000,1500.000000,1542.000000,1437.000000,1545.000000,1465.000000,1491.000000,1546.000000,1524.000000,0.517067,15000.000000,15000.000000


# Radial basis function SVM

In [11]:
%%time
rbfsvclassifier = SVC(kernel='rbf', tol=0.002)
rbfsvclassifier.fit(X_train, y_train)

Wall time: 58min 46s


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.002, verbose=False)

In [12]:
# avalia predicao
y_pred = rbfsvclassifier.predict(X_test)
report = classification_report(y_test, y_pred,output_dict = True)
pd.DataFrame(report)

,airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck,accuracy,macro avg,weighted avg
f1-score,0.622578,0.663833,0.469619,0.392224,0.483274,0.469346,0.615527,0.605580,0.725260,0.652906,0.572667,0.570015,0.569822
precision,0.612755,0.641999,0.484076,0.413669,0.494538,0.495327,0.562783,0.622521,0.730013,0.632615,0.572667,0.569030,0.569074
recall,0.632722,0.687204,0.456000,0.372892,0.472512,0.445955,0.679181,0.589537,0.720569,0.674541,0.572667,0.573111,0.572667
support,1473.000000,1477.000000,1500.000000,1542.000000,1437.000000,1545.000000,1465.000000,1491.000000,1546.000000,1524.000000,0.572667,15000.000000,15000.000000
